In [2]:
import pyzed.sl as sl
# 创建ZED相机对象
zed = sl.Camera()

# 设置初始化参数
init_params = sl.InitParameters()
init_params.camera_resolution = sl.RESOLUTION.HD720  # 设置分辨率
init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # 设置深度模式
init_params.coordinate_units = sl.UNIT.METER  # 深度单位为米

# 打开相机
status = zed.open(init_params)
if status != sl.ERROR_CODE.SUCCESS:
    print(f"无法打开相机: {status}")
    exit(1)

# 创建图像和深度图像对象
image = sl.Mat()
depth = sl.Mat()


libEGL warning: DRI2: failed to authenticate


[2025-07-11 03:22:25 UTC][ZED][INFO] Logging level INFO
[2025-07-11 03:22:26 UTC][ZED][WARNING] [Init]  ULTRA is deprecated; consider updating to a NEURAL depth mode
[2025-07-11 03:22:27 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-07-11 03:22:27 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-07-11 03:22:27 UTC][ZED][INFO] [Init]  Video mode: HD720@30
[2025-07-11 03:22:27 UTC][ZED][INFO] [Init]  Serial Number: S/N 35269462


In [7]:
# 捕获图像
if zed.grab() == sl.ERROR_CODE.SUCCESS:
    # 获取左眼图像
    zed.retrieve_image(image, sl.VIEW.LEFT)
    # 获取深度图
    zed.retrieve_measure(depth, sl.MEASURE.DEPTH)

    # 获取某个像素点的深度值 (例如图像中心点)
    width = depth.get_width()
    height = depth.get_height()
    center_x = width // 2
    center_y = height // 2

    err, depth_value = depth.get_value(center_x, center_y)
    if err == sl.ERROR_CODE.SUCCESS:
        print(f"中心点深度: {depth_value:.2f} 米")
    else:
        print(f"无法获取深度值: {err}")

中心点深度: 3.90 米


In [21]:
import numpy as np  # 确保已导入 numpy

# 捕获图像
if zed.grab() == sl.ERROR_CODE.SUCCESS:
    # 获取深度图
    zed.retrieve_measure(depth, sl.MEASURE.DEPTH)
    # 将深度数据转换为 numpy 数组
    depth_numpy = depth.get_data()  # 假设 depth.numpy() 返回 numpy 数组

    # 获取深度矩阵的最小值（忽略 NaN）
    min_depth = np.nanmin(depth_numpy)
    print(f"深度矩阵的最小值: {min_depth:.2f} 米")

深度矩阵的最小值: 0.41 米


In [51]:
# ...existing code...

import numpy as np  # 确保已导入 numpy

# 捕获图像
if zed.grab() == sl.ERROR_CODE.SUCCESS:
    # 获取深度图
    zed.retrieve_measure(depth, sl.MEASURE.DEPTH)
    # 将深度数据转换为 numpy 数组
    depth_numpy = depth.get_data()  # 假设 depth.get_data() 返回 numpy 数组

    # 过滤无效值（例如 -inf, inf）
    depth_numpy = np.where(np.isfinite(depth_numpy), depth_numpy, np.nan)

    # 获取深度矩阵的最小值（忽略 NaN）
    if np.isnan(depth_numpy).all():
        print("深度矩阵中没有有效值")
    else:
        min_depth = np.nanmin(depth_numpy)
        print(f"深度矩阵的最小值: {min_depth:.2f} 米")

# ...existing code...

深度矩阵的最小值: 1.30 米


In [32]:
# 关闭相机
zed.close()